In [1]:
# some code implemented from youtube tutorial from Madhav Thaker
# https://github.com/madhavthaker/spotify-recommendation-system

In [2]:
import pandas as pd
import numpy as np
import json
from IPython.display import Image
from IPython.core.display import display, HTML
import itertools
import sys
import re
import os
import matplotlib.pyplot as plt
from skimage import io
from IPython.display import Image

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

url = os.getcwd()

In [3]:
from secrets import client_id, client_secret

scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

# Spotify access
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)


spotify_username = sp.current_user()['id']

In [4]:
def get_users_playlists():
    playlist_names = {}
    playlist_photos = {}
    
    for i in sp.current_user_playlists()['items']:
        playlist_names[i['name']] = i['uri'].split(':')[2]
        playlist_photos[i['uri'].split(':')[2]] = i['images'][0]['url']
    
    return playlist_names, playlist_photos

# Call get_users_playlists()
playlist_names, playlist_photos = get_users_playlists()

In [5]:
playlist_names

{'New Playlist 2': '5e8ijEDzlaB0y9apSoZNBI',
 'Tyler Bates – The Punisher (Original Soundtrack) 2': '3qt3t2qfyWSli9F6C9C3SF',
 'Globus – Epicon (UK Version)': '25sJ1ieBzb2AVPS9GaZhKm',
 's': '2L0LTJqWOFFI8qPqkDVjKG',
 'Immediate – Trailerhead: Triumph': '71DD8rTMqJB4bnFRdS1Jz7',
 'Electric Romeo – Immediate': '2aL9qUB2A6vc72I4AwrWlt',
 'Highway man': '4thvxQYvACp32gZguRAiEw',
 'party mood': '7MIO3PrQ0ZXdPobIpHOc5a',
 'Southpaw (Music From And Inspired By The Motion Picture)': '0kF5WxQpwIlFG130bWovut',
 'Work Hard, Play Hard – Wiz Khalifa': '1CavHtskIP3Dp0r9RCwXpY',
 'From edits': '5Yi5I6uqMn9oyV3RhvUVNm',
 'beuty': '7K0flcn5YDOXxPRhChKzre',
 'OAZ': '0RVTSPELkBvNXUw27BUC0R',
 'Eyeex': '2zqTk2BwhJx4CaqhwZ40RK',
 'Icelandic Music | Ragnarsmedia': '2eyFd9OYGYRFG1WknFYfTy',
 'Íslenskt': '4jp0RPWEON6B3hbLu6f1N4',
 'Grime ': '3VsZqGKOP5E6pcixUKg8zt',
 'Ultra 1.0': '0fQrVALjgMV5ckiOG6Lw0u',
 'Ultra 2.0': '6N1CYumwsQ6hScprqxmfsL',
 'Ultra 3.0': '1thpoCGLWqyLpPMFli6dHD',
 '21': '3WCO6yNpxaBEtRVT

In [6]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [25]:
def get_track_info(playlist_id,tracks, number):
    # Track
    track = tracks[number]
    added_by = track['added_by']['id']
    track_info = track['track']
    track_artist = track_info['artists'][0]
    track_album = track_info['album']

    # Informations about the track
    id = track_info['id']
    name = track_info['name']
    date_added = track['added_at']
    explicit = track_info['explicit']
    popularity = track_info['popularity']
    #print(f"{id} + {name}")
    if( 'spotify' in track_info):
        link = ""
    else:
        link = track_info['external_urls']['spotify']
    preview_url = track_info['preview_url']

    # Informations about the artists
    artist_id = track_artist['id']
    artist_name = track_artist['name']
    
    # get tracks album image
    image_url = track_album['images'][1]['url']
    
    
    track_out = {
        'playlist_id': playlist_id,
        'id': id,
        'name': name,
        'link': link,
        'popularity': popularity,
        'image_url': image_url,
        'preview_url': preview_url,
        'artist_id': artist_id,
        'artist_name': artist_name,
        'explicit': explicit,
        'date_added': date_added,
    }
    return track_out

In [26]:
def specific_playlist_songs(id_playlist):
    track_info = []
    for playlist_name, playlist_id in playlist_names.items():
        if(str(playlist_id) == id_playlist):
            tracks = get_playlist_tracks(spotify_username, playlist_id)
            for track_number in range(0, len(tracks)):
                track_info.append(get_track_info(playlist_id, tracks, track_number))
    return pd.DataFrame(track_info)

def get_all_playlist_songs():
    track_info = []
    for playlist_name, playlist_id in playlist_names.items():
        tracks = get_playlist_tracks(spotify_username, playlist_id)
        for track_number in range(0, len(tracks)):
            track_info.append(get_track_info(playlist_id, tracks, track_number))
    return pd.DataFrame(track_info)    


In [28]:
from_edits_playlist = specific_playlist_songs("5Yi5I6uqMn9oyV3RhvUVNm")
from_isx_playlist = specific_playlist_songs("4jp0RPWEON6B3hbLu6f1N4")

#from_edits_playlist

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])
dict_keys(['album', 'artists', '

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def cover_songs(df):
    """ 
    Visualize cover art of the songs in the inputted dataframe

    Parameters: 
        df (pandas dataframe): Playlist Dataframe
    """
    
    image_url = df['image_url'].values
    plt.figure(figsize=(15,int(0.625 * len(image_url))))
    columns = 5
    
    for i, url in enumerate(image_url):
        plt.subplot(len(image_url) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
cover_songs(from_edits_playlist)

In [ ]:
cover_songs(from_isx_playlist)

In [ ]:
## show all recommendations

In [ ]:
## show bad recommendations (low popularity)

In [ ]:
## show good recommendations (high popularity)